In [ ]:
import os
from itertools import islice
from pprint import pprint

import vk_api
from vk_api.audio import VkAudio

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

### VK credentials
Set your login and password for VK through environment variables `VK_ACCOUNT_LOGIN` and `VK_ACCOUNT_PASSWORD` accordingly before running jupyter notebook. Alternatively set them in the next cell manually. 

In [ ]:
ACCOUNT_LOGIN = os.getenv('VK_ACCOUNT_LOGIN')
ACCOUNT_PASSWORD = os.getenv('VK_ACCOUNT_PASSWORD')

In [ ]:
vk_session = vk_api.VkApi(ACCOUNT_LOGIN, ACCOUNT_PASSWORD)
vk_session.auth()
vk_audio = VkAudio(vk_session)
vk_tracks = vk_audio.get_iter()

In [ ]:
vk_tracks_list = list(vk_tracks)

### Spotify credentials
Go to the https://developer.spotify.com/dashboard/ and create App. Get the `Client ID` and `Client Secret`, then set `Redirect URIs` as your `APP_REDIRECT_URI` - it could be just http://localhost:8000. `APP_SCOPE` is used to grant permissions to your App.

In [ ]:
APP_CLIENT_ID = 'your-app-client-id'
APP_CLIENT_SECRET = 'your-app-client-secret'
APP_REDIRECT_URI = 'http://localhost:8000'
APP_SCOPE = 'user-read-currently-playing, user-read-private, playlist-modify-private, playlist-read-private, user-library-read'

In [ ]:
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=APP_CLIENT_ID,
        client_secret=APP_CLIENT_SECRET,
        redirect_uri=APP_REDIRECT_URI,
        scope=APP_SCOPE
    )
)

#### Your saved albums

In [ ]:
result = sp.current_user_saved_albums()
for idx, item in enumerate(result['items']):
    album = item['album']
    print(idx, f'{album["artists"][0]["name"]} – {album["name"]}')

#### Your saved playlists

In [ ]:
result = sp.current_user_playlists()
for idx, item in enumerate(result['items']):
    print(
        idx, 
        (
            f'{item["name"]} '
            f'– owner: {item["owner"]["display_name"]} '
            f'– id: {item["id"]} '
            f'– total tracks: {item["tracks"]["total"]}'
        )
    )

### Transfer VK music to Spotify
Try to find exact track on Spotify. Check the title and the artist in lowercase.

In [ ]:
not_found_tracks = []
spotify_tracks = []

for vk_track in vk_tracks_list:
    spotify_track = None
    try:
        tracks_found = sp.search(q=f"{vk_track['title']}  {vk_track['artist']}")
        for item in tracks_found['tracks']['items']:
            if (
                str.lower(item['name']) == str.lower(vk_track['title']) 
                and str.lower(item['artists'][0]['name']) == str.lower(vk_track['artist'])
            ):
                spotify_track = {
                    'id': item['id'],
                    'name': item['name'],
                    'artist': item['artists'][0]['name'],
                    'album': item['album']['name']
                } 
                break
    except Exception as e:
        print(e)
    if spotify_track is None:
        not_found_tracks.append({'title': vk_track['title'], 'artist': vk_track['artist']})
    else:
        spotify_tracks.append(spotify_track)

In [ ]:
print(f'Total found tracks on VK: {len(vk_tracks_list)}\n\nNot found tracks from VK: {len(not_found_tracks)}\n')
pprint(not_found_tracks)
print(f'\nFound track on Spotify: {len(spotify_tracks)}\n')
pprint(spotify_tracks)

In [ ]:
def batched(iterable, batch_size):
    """ Split iterable by batches
    >>> list(batched(range(10), batch_size=3))
    [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9]]
    """
    iterator = iter(iterable)
    while True:
        objects = list(islice(iterator, batch_size))
        if not objects:
            break
        yield objects

### Add found track to your playlist on Spotify
Create a new playlist for your VK music or use already existed one. Just get the id from `Your saved playlists` cell and set it to `VK_PLAYLIST` variable.

In [ ]:
VK_PLAYLIST = 'your-playlist-id'

for batch in batched([track['id'] for track in spotify_tracks], 100):
    result = sp.playlist_add_items(VK_PLAYLIST, batch)